<h4> The number of columns in the dataset encourages to try embedding and it gives quite good result.
    The onehot encoding leads to around 3300 columns and don't give good results.
    The vertical embedding (by columns) is a bit disapointing but needs more investigation.
    This notebook don't perform any features engineering, so it could be a way of improvement.
    There are no missing values and no negative values.
   

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn import preprocessing
from sklearn.metrics import log_loss

import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import activations,callbacks
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy

from keras.models import Model

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-jun-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-jun-2021/test.csv')

In [ ]:
target = pd.get_dummies(train['target'])
y = train['target']

In [ ]:
X = train.iloc[:,1:-1]
test = test.iloc[:,1:]
X.shape, test.shape

In [ ]:
def api_func_embedding():
    inputs_API_Embedding = layers.Input(shape = (75,), name = 'API_input_Embedding')
    x = layers.Embedding(400, 10, input_length = 256)(inputs_API_Embedding)
    x = layers.Flatten()(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(100, activation='relu')(x)
    x = layers.Dense(50, activation='relu')(x)
    x = layers.Dense(18, activation = 'sigmoid')(x)
    out = layers.Dense(9, activation = 'softmax', name = 'out')(x)
    model = Model (inputs_API_Embedding,out)
    
    return model

In [ ]:
es = callbacks.EarlyStopping(
                monitor = 'val_categorical_crossentropy', 
                min_delta = 0.0000001, 
                patience = 2,
                mode = 'min',
                baseline = None, 
                restore_best_weights = True,
                verbose = 1)

plateau  = callbacks.ReduceLROnPlateau(
                monitor = 'val_categorical_crossentropy',
                factor = 0.5, 
                patience = 2, 
                mode = 'min', 
                min_delt = 0.0000001,
                cooldown = 0, 
                min_lr = 1e-7,
                verbose = 1) 

metrics = [tf.keras.metrics.CategoricalCrossentropy()]
loss = tf.keras.losses.CategoricalCrossentropy(
                from_logits=False,
                label_smoothing=0,
                reduction="auto",
                name="categorical_crossentropy")

In [ ]:
N_FOLDS = 10
SEED = 2021
oof = np.zeros((X.shape[0],9))
pred = np.zeros((test.shape[0],9))


skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)

for fold, (tr_idx, ts_idx) in enumerate(skf.split(X, y)):
    print(f"===== FOLD {fold} =====")
       
    x_tr = X.iloc[tr_idx] 
    y_tr = target.iloc[tr_idx] 
    
    x_ts = X.iloc[ts_idx] 
    y_ts = target.iloc[ts_idx] 

    model_nn= api_func_embedding()

    model_nn.compile(tf.keras.optimizers.Adam(learning_rate=0.0001),
            loss = loss ,
            metrics = metrics)
    
    model_nn.fit(
                    x_tr,
                    y_tr,
                    validation_data = (
                    x_ts,
                    y_ts),
                    batch_size = 256,
                    epochs = 50,
                    verbose = 1,
                    callbacks = [es,plateau])
    
    
    oof[ts_idx] = model_nn.predict(x_ts)

    score = log_loss(y_ts, oof[ts_idx])
    print(f"FOLD {fold} Score {score}\n")
    
    pred += model_nn.predict(test)/ N_FOLDS

score = log_loss(target, oof)
print(f"Score total {score}\n")  

In [ ]:
submission = pd.read_csv("../input/tabular-playground-series-jun-2021/sample_submission.csv")

In [ ]:
pred_embedding = pred

In [ ]:

submission['Class_1']=pred_embedding[:,0]
submission['Class_2']=pred_embedding[:,1]
submission['Class_3']=pred_embedding[:,2]
submission['Class_4']=pred_embedding[:,3]
submission['Class_5']=pred_embedding[:,4]
submission['Class_6']=pred_embedding[:,5]
submission['Class_7']=pred_embedding[:,6]
submission['Class_8']=pred_embedding[:,7]
submission['Class_9']=pred_embedding[:,8]
submission.head()

In [ ]:
submission.to_csv("Keras_embedding.csv", index=False)